In [1]:
import os,sys
sys.path.append("../notebook_utilities/")
from analysis_common import *

In [2]:
from root_common import *

Welcome to JupyROOT 6.30/02


In [3]:
import pandas as pd
import dask.dataframe as dd

In [4]:
import glob

In [5]:
# Pattern to match the ROOT files
file_pattern = "/Users/wketchum/Data/LDMX/production_07Jan2024/*_ana.root"

# List to hold the DataFrames
dfs = []

In [6]:
# Loop through each file matching the pattern
for filename in glob.glob(file_pattern)[:10]:
    print(filename)
    # Open the ROOT file
    with uproot.open(filename) as file:
        # Extract the TTree named "ana_tree"
        tree = file["ana_tree"]
        
        df = tree.arrays(library="pd")
        
        # Convert Pandas DataFrame to Dask DataFrame
        ddf = dd.from_pandas(df, npartitions=1)  # Adjust npartitions based on your dataset/cluster
        dfs.append(ddf)

# Concatenate all Dask DataFrames into a single DataFrame
combined_ddf = dd.multi.concat(dfs, axis=0)

# Concatenate all DataFrames into a single DataFrame
#combined_df = pd.concat(dfs, ignore_index=True)

/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101228_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101074_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101124_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101282_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101063_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101133_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101211_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101206_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_101286_reco_ana.root
/Users/wketchum/Data/LDMX/production_07Jan2024/ldmx_genie_G18_02a_02_11b_Ti_8GeV_1

TypeError: Cannot interpret '<AwkwardDtype>' as a data type

In [ ]:
combined_df = pd.concat(dfs[:2],ignore_index=True)

In [ ]:
# Generating logarithmically spaced bins
x_bins = np.logspace(np.log10(0.1), np.log10(10), 40)
y_bins = np.logspace(np.log10(0.01), np.log10(20), 40)

In [ ]:
%matplotlib inline

In [ ]:
combined_df["q_2"] = combined_df["elec_px"]*combined_df["elec_px"]+combined_df["elec_py"]*combined_df["elec_px"]+(8000-combined_df["elec_pz"])*(8000-combined_df["elec_pz"])

In [ ]:
combined_df["Q2"] = combined_df["q_2"]-(8000-combined_df["elec_e"])*(8000-combined_df["elec_e"])

In [ ]:
combined_df["w"] = (8000-combined_df["elec_e"])

In [ ]:
combined_df

In [ ]:
filtered_df = combined_df.query("elec_pt>500 and genie_dis==1")

# Creating the 2D histogram plot
plt.figure(figsize=(10, 7))
plt.hist2d(filtered_df["w"]/1e3, filtered_df["Q2"]/1e6, bins=[x_bins, y_bins], cmap='viridis')
plt.colorbar(label='Counts in bin')

# Setting the scale to logarithmic if necessary
plt.xscale('log')
plt.yscale('log')

# Setting labels and title
plt.xlabel('Energy Transfer (GeV)')
plt.ylabel('Q2 (GeV2)')
plt.title('Momentum transfer vs Energy Transfer for 8 GeV eN interactions')

# Display the plot
plt.show()

In [ ]:
filtered_df = combined_df.query("elec_pt>500 and genie_res==1")

# Creating the 2D histogram plot
plt.figure(figsize=(10, 7))
plt.hist2d(filtered_df["w"]/1e3, filtered_df["Q2"]/1e6, bins=[x_bins, y_bins], cmap='viridis')
plt.colorbar(label='Counts in bin')

# Setting the scale to logarithmic if necessary
plt.xscale('log')
plt.yscale('log')

# Setting labels and title
plt.xlabel('Energy Transfer (GeV)')
plt.ylabel('Q2 (GeV2)')
plt.title('Momentum transfer vs Energy Transfer for 8 GeV eN interactions')

# Display the plot
plt.show()

In [ ]:
ana_files = glob.glob('/Users/wketchum/Data/LDMX/production_07Jan2024/*_ana.root')
ana_files.sort()

In [ ]:
ana_tree = ROOT.TChain("ana_tree")

In [ ]:
ana_tree.Add('/Users/wketchum/Data/LDMX/production_07Jan2024/*_ana.root')

In [ ]:
c = ROOT.TCanvas("c")

In [ ]:
for obj in ana_tree.GetListOfBranches():
    print(obj)

In [ ]:
ana_tree.Draw("elec_pt")
c.Draw()

In [ ]:
ana_tree.Draw("(elec_px*elec_px)+(elec_py*elec_py)+(8000-elec_pz)*(8000-elec_pz)","elec_pt>500")
c.Draw()

In [ ]:
ana_tree.Draw("elec_e:elec_pt","1","colz")
c.Draw()

In [ ]:
ana_tree.Draw("((elec_px*elec_px)+(elec_py*elec_py)+(8000-elec_pz)*(8000-elec_pz)-(8000-elec_e)*(8000-elec_e))/1e6:(8000-elec_e)/1e3","1","colz")
c.Draw()